## QPTH
Differentiating sum of solution using optnet

In [68]:
from qpth.qp import QPFunction
import torch
import numpy as np
from torch import nn
from torch.autograd import Variable

In [69]:
Q = np.array([[2., 1., 0.],
              [1., 2., 1.],
              [0., 1., 2.]])
q = np.array([0., 0., 0.])
G = np.array([[-1., -2., -3.], [-1., -1., 0.]])
h = np.array([-4., -1.])

In [70]:
q, Q, G, h = [torch.Tensor(x) for x in [q, Q, G, h]]

A, b = [torch.Tensor()] * 2

q, Q, G, h, A, b = [Variable(x) for x in [q, Q, G, h, A, b]]
Q.requires_grad = True
q.requires_grad = True
G.requires_grad = True
h.requires_grad = True

In [71]:
qpf = QPFunction()
zhat_b = qpf(Q, q, G, h, A, b)

In [72]:
zhat_b  # solution

tensor([[0.5714, 0.4286, 0.8571]], grad_fn=<QPFunctionFnBackward>)

In [73]:
# zhat_b.backward(torch.Tensor([[1.1, 0.9, 0.8]]))
zhat_b.backward(torch.Tensor([[1.0, 1.0, 1.0]])) # setting dl/dz = ones(3,1)

In [74]:
grads = [x.grad.data.squeeze(0).cpu().numpy() for x in [Q, q, G, h]]

In [75]:
dQ = grads[0]
dQ

array([[-0.12244895,  0.01530609, -0.11224488],
       [ 0.01530609,  0.09183674,  0.07653058],
       [-0.11224488,  0.07653058, -0.06122449]], dtype=float32)

In [76]:
dq = grads[1]
dq

array([-0.2142857 ,  0.21428567, -0.07142857], dtype=float32)

In [77]:
dG = grads[2]
dG

array([[0.05102035, 0.30612245, 0.255102  ],
       [0.06122443, 0.36734694, 0.3061224 ]], dtype=float32)

In [78]:
dh = grads[3]
dh

array([-0.35714284, -0.4285714 ], dtype=float32)

## CVXPYLayers
Differentiating using cvxpylayers. Note that we can't differentiate w.r.t `Q` matrix as defining it as a parameter violates DPP constraints

In [88]:
import numpy as np
import cvxpy as cp
import tensorflow as tf
from cvxpylayers.tensorflow import CvxpyLayer

In [81]:
n = 3
m = 2

x = cp.Variable(n)
Q = np.array([[2., 1., 0.],
              [1., 2., 1.],
              [0., 1., 2.]])
_q = np.array([0., 0., 0.])
_G = np.array([[-1., -2., -3.], [-1., -1., 0.]])
_h = np.array([-4., -1.])
q = cp.Parameter(n)
G = cp.Parameter((m, n))
h = cp.Parameter(m)

In [82]:
constraints = [G@x <= h]
objective = cp.Minimize(0.5*cp.quad_form(x, Q) + q.T @ x)
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

# cvxpylayers computes gradient of sum of solution wrt params
# i.e. u don't need to pass dl/dz, its fixed as tf.ones(n)

cvxpylayer = CvxpyLayer(problem, parameters=[q, G, h], variables=[x])

In [83]:
q_tf = tf.Variable(
    tf.convert_to_tensor(
        _q, 
        np.float32
    )
)

h_tf = tf.Variable(
    tf.convert_to_tensor(
        _h, 
        np.float32
    )
)

G_tf = tf.Variable(
    tf.convert_to_tensor(
        _G, 
        np.float32
    )
)

In [84]:
with tf.GradientTape() as tape:
  solution, = cvxpylayer(q_tf, G_tf, h_tf)
  summed_solution = tf.math.reduce_sum(solution)
    
grads = tape.gradient(summed_solution, [q_tf, G_tf, h_tf])

In [85]:
grads[0]

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([-0.21428571,  0.21428571, -0.07142857])>

In [86]:
grads[1]

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[0.05102041, 0.30612245, 0.25510204],
       [0.06122449, 0.36734694, 0.30612245]])>

In [87]:
grads[2]

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([-0.35714286, -0.42857143])>